# Mean-state changes
Look at how mean state changes over time

In [ ]:
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import os
import src.XRO
import copy
import scipy.stats
import warnings
import calendar

# import gsw

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## Funcs

In [ ]:
def get_ensemble_fn(data, fn):
    """apply python function to data using xr.apply_ufunc"""

    ## stack data
    data_stack = data.stack(sample=["time", "member"])

    ## kwargs for apply_ufunc
    kwargs = dict(
        input_core_dims=[["sample", "year"]],
        output_core_dims=[["year"]],
    )

    return xr.apply_ufunc(fn, data_stack, **kwargs)


def get_ensemble_fn_bymonth(data, fn):
    """apply python function to data by month using xr.apply_ufunc"""
    return data.groupby("time.month").map(get_ensemble_fn, fn=fn)


def get_nino3(x, z_t=70, lat_bound=5):
    """get Niño 3 average, with checks in place"""

    if "z_t" in x.dims:
        x = x.sel(z_t=z_t, method="nearest")

    if "latitude" in x.dims:
        x = x.sel(latitude=slice(-lat_bound, lat_bound)).mean("latitude")

    return x.sel(longitude=slice(210, 270)).mean("longitude")


def get_nino3_helper(x, lat_bounds):
    """get Niño 3 average, with checks in place"""

    ## get index to select data
    idx = dict(latitude=slice(*lat_bounds), longitude=slice(210, 270))

    return x.sel(idx).mean(["longitude", "latitude"])


def get_nino3_north(x):
    return get_nino3_helper(x, lat_bounds=(0, 10))


def get_nino3_south(x):
    return get_nino3_helper(x, lat_bounds=(-10, 0))

## Load data

In [ ]:
## load spatial data
forced, anom_ = src.utils.load_consolidated()

## get subset of data for total
VARNAMES = ["T", "w", "pr", "tauy"]
total = anom_[VARNAMES] + forced[VARNAMES]
total = xr.merge([forced[[f"{v}_comp" for v in VARNAMES]], total])

## get nino3 averages
total_n3 = src.utils.reconstruct_wrapper(total, fn=get_nino3)

## rename to avoid conflicts
total_n3 = total_n3.rename({v: f"{v}_n3" for v in VARNAMES})

## get precip North and south of equator
pr_N = src.utils.reconstruct_wrapper(
    total[["pr", "pr_comp", "tauy", "tauy_comp"]],
    fn=get_nino3_north,
).rename({"pr": "pr_n", "tauy": "tauy_n"})

pr_S = src.utils.reconstruct_wrapper(
    total[["pr", "pr_comp", "tauy", "tauy_comp"]],
    fn=get_nino3_south,
).rename({"pr": "pr_s", "tauy": "tauy_s"})

## merge
total_n3 = xr.merge([total_n3, pr_N, pr_S])

In [ ]:
## get windowed data
forced = src.utils.get_windowed(forced).groupby("time.month").mean()

## reconstruct data
forced = src.utils.reconstruct_wrapper(forced)

# ## load tropical SST avg
trop_sst = xr.open_dataset(pathlib.Path(DATA_FP, "cesm/trop_sst.nc"))

## Load T,h (total)
Th_total = xr.open_dataset(DATA_FP / "cesm" / "Th.nc")
Th_total = xr.merge([Th_total, trop_sst])

## custom h data
h_mg_forced, h_mg_anom = src.utils.load_h_data(max_grad=True)

In [ ]:
## compute dTdx
Th_total["dTdx"] = Th_total["T_4"] - Th_total["T_3"]

## load ELI
eli = xr.open_dataset(pathlib.Path(DATA_FP, "cesm/eli.nc"))
Th_total = xr.merge([Th_total, eli, total_n3])


## windowed version
Th_total = src.utils.get_windowed(Th_total)

## relative SST
for v in ["T_3", "T_34", "T_4"]:
    Th_total[f"{v}_rel"] = Th_total[v] - Th_total["trop_sst_15"]

## ensemble mean
Th_total_forced = Th_total.groupby("time.month").mean(["time", "member"])

## merge data
forced = xr.merge([forced, Th_total_forced])

In [ ]:
## compute quantiles
quant = Th_total.groupby("time.season").quantile(
    q=[0.05, 0.25, 0.5, 0.75, 0.95], dim=["time", "member"]
)
quant = quant.rename({"quantile": "q"})

## compute difference
delta_q = quant - quant.isel(year=0)

## get spread
spread = quant.sel(q=0.95) - quant.sel(q=0.05)
delta_spread = spread - spread.isel(year=0)

#### Compute variance

In [ ]:
## open data
Th = src.utils.load_cesm_indices()

## get windowed
Th = src.utils.get_windowed(Th)
for v in ["05", "15", "20", "25", "30"]:
    Th[f"eli_{v}"] = Th_total[f"eli_{v}"] - Th_total[f"eli_{v}"].mean("member")

## get variance by month
Th_sigma = Th.groupby("time.month").std(["time", "member"])

## fractional change
Th_delta_sigma = (Th_sigma - Th_sigma.isel(year=0)) / Th_sigma.isel(year=0)

#### Compute skewness

In [ ]:
Th_skew = get_ensemble_fn_bymonth(Th, fn=scipy.stats.skew)
Th_delta_skew = Th_skew - Th_skew.isel(year=0)

#### Compute quantiles

In [ ]:
## compute quantiles
quant = Th_total.groupby("time.season").quantile(
    q=[0.02, 0.05, 0.15, 0.25, 0.5, 0.75, 0.85, 0.95, 0.98], dim=["time", "member"]
)
quant = quant.rename({"quantile": "q"})

## compute difference
delta_q = quant - quant.isel(year=0)

## get spread
spread = quant.sel(q=0.95) - quant.sel(q=0.05)
delta_spread = spread - spread.isel(year=0)

## Analysis

### Spatial plots

In [ ]:
forced0 = forced.sel(year=1870)
forced1 = forced.sel(year=1980)
forced2 = forced.sel(year=2085)

#### SST

In [ ]:
v0 = "sst"
v1 = "trop_sst_15"

import cartopy.crs as ccrs

sel_ = lambda x: x.sel(month=[12,1,2]).mean("month")

fig = plt.figure(figsize=(10, 4.375), layout="constrained")
format_func = lambda ax: src.utils.plot_setup_pac(ax, max_lat=30)
axs = src.utils.subplots_with_proj(fig, nrows=3, ncols=2, format_func=format_func)

## specify kwargs
plot_kwargs = dict(
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(10, 1),
    extend="both",
    transform=ccrs.PlateCarree(),
)

for i, f, f_ in zip(
    np.arange(3), [forced0, forced1, forced2], [forced0, forced0, forced1]
):

    ## plot relative SST
    axs[i, 0].contourf(
        f.longitude,
        f.latitude,
        sel_(f[v0] - f[v1]),
        **plot_kwargs,
    )

    ## difference
    axs[i, 1].contourf(
        f.longitude,
        f.latitude,
        5 * (sel_(f[v0] - f[v1]) - sel_(f_[v0] - f_[v1])),
        **plot_kwargs,
    )


plt.show()

#### precip

In [ ]:
v0 = "pr"

import cartopy.crs as ccrs

sel_ = lambda x: x.sel(month=[12,1,2]).mean("month")

fig = plt.figure(figsize=(10, 4.375), layout="constrained")
format_func = lambda ax: src.utils.plot_setup_pac(ax, max_lat=30)
axs = src.utils.subplots_with_proj(fig, nrows=3, ncols=2, format_func=format_func)

## specify kwargs
plot_kwargs = dict(
    extend="both",
    transform=ccrs.PlateCarree(),
)

for i, f, f_ in zip(
    np.arange(3), [forced0, forced1, forced2], [forced0, forced0, forced1]
):

    ## plot relative SST
    axs[i, 0].contourf(
        f.longitude,
        f.latitude,
        8.6e4 * sel_(f[v0]),
        cmap="cmo.rain",
        levels=np.arange(0, 16, 2),
        **plot_kwargs,
    )

    ## difference
    axs[i, 1].contourf(
        f.longitude,
        f.latitude,
        8.6e4 * (sel_(f[v0] - f_[v0])),
        cmap="cmo.balance_r",
        levels=src.utils.make_cb_range(4, 0.4),
        **plot_kwargs,
    )


plt.show()

#### wind

In [ ]:
# ## variable to plot
# v0 = "taux"
# SC=8e1

## variable to plot
v0 = "tauy"
SC = -16e1

import cartopy.crs as ccrs

sel_ = lambda x: x.sel(month=[3]).mean("month")

fig = plt.figure(figsize=(10, 4.375), layout="constrained")
format_func = lambda ax: src.utils.plot_setup_pac(ax, max_lat=30)
axs = src.utils.subplots_with_proj(fig, nrows=3, ncols=2, format_func=format_func)

## specify kwargs
plot_kwargs = dict(
    extend="both",
    transform=ccrs.PlateCarree(),
)

for i, f, f_ in zip(
    np.arange(3), [forced0, forced1, forced2], [forced0, forced0, forced1]
):

    ## plot relative SST
    axs[i, 0].contourf(
        f.longitude,
        f.latitude,
        SC * sel_(f[v0]),
        cmap="cmo.balance",
        levels=src.utils.make_cb_range(10, 1),
        **plot_kwargs,
    )

    ## difference
    axs[i, 1].contourf(
        f.longitude,
        f.latitude,
        SC * (sel_(f[v0] - f_[v0])),
        cmap="cmo.balance",
        levels=src.utils.make_cb_range(2, 0.2),
        **plot_kwargs,
    )

    axs[i, 1].axhline(0, ls="--", c="w")


plt.show()

### Precip / $\tau_x$ on the equator

In [ ]:
## specify month and lat bound for averaging
MONTH = 5
LAT_BOUND = 5

## func to select data
merimean = lambda x, lat_bound=5: x.sel(latitude=slice(-lat_bound, lat_bound)).mean(
    "latitude"
)
sel = lambda x: merimean(x, lat_bound=LAT_BOUND).sel(month=MONTH).transpose("year", ...)

## get longitude for convenience
LON = forced.longitude

## compute center of mass
forced_pac = forced.sel(longitude=slice(140, 280))
com = sel(forced_pac * LON).integrate("longitude") / sel(forced_pac).integrate(
    "longitude"
)

## get argmin
taux_argmin = forced_pac.longitude.isel(
    longitude=sel(forced_pac["taux"]).argmin("longitude")
)

fig, axs = plt.subplots(1, 2, figsize=(4, 4), layout="constrained")

## precip
axs[0].contourf(
    forced.longitude,
    forced.year,
    8.6e4 * sel(forced["pr"]),
    cmap="cmo.rain",
    levels=np.arange(0, 13.5, 1.5),
)

## wind
axs[1].contourf(
    forced.longitude,
    forced.year,
    sel(forced["taux"]),
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(0.075, 0.0075),
)

## plot longitude of max grad
axs[1].plot(
    taux_argmin,
    forced.year,
    c="w",
    ls="--",
    alpha=1,
    lw=1,
)

## plot longitude of max grad
axs[0].plot(
    com["pr"],
    forced.year,
    c="w",
    ls="--",
    alpha=1,
    lw=1,
)


## formatting
for ax in axs:
    ax.set_xlim([140, 280])
for ax in axs[1:]:
    ax.set_yticks([])

plt.show()

In [ ]:
## specify month and lat bound for averaging
MONTH = 2

## func to select data
zonamean = lambda x: x.sel(longitude=slice(210, 270)).mean("longitude")
sel = lambda x: zonamean(x).sel(month=[2, 3, 4]).mean("month").transpose("year", ...)


fig, axs = plt.subplots(1, 2, figsize=(4, 4), layout="constrained")

## precip
axs[0].contourf(
    forced.latitude,
    forced.year,
    8.6e4 * sel(forced["pr"]),
    cmap="cmo.rain",
    levels=np.arange(0, 10, 1),
    extend="both",
)

## wind
axs[1].contourf(
    forced.latitude,
    forced.year,
    -sel(forced["tauy"]) + sel(forced["tauy"]).isel(year=0),
    cmap="cmo.balance",
    # levels=src.utils.make_cb_range(5e-2, 5e-3),
    levels=src.utils.make_cb_range(1e-2, 1e-3),
    extend="both",
)

## formatting
for ax in axs:
    ax.axvline(0, c="w", ls="--", lw=0.8)
    ax.set_xlim([-15, 15])
for ax in axs[1:]:
    ax.set_yticks([])

plt.show()

In [ ]:
y = 8.6e4 * forced["pr"].sel(longitude=slice(210, 270)).mean("longitude")


fig, axs = plt.subplots(2, 1, figsize=(5, 6))
axs[0].contourf(
    y.month,
    y.latitude,
    y.sel(year=1870).transpose("latitude", ...),
    cmap="cmo.rain",
    levels=np.arange(0, 12, 1),
    extend="both",
)

axs[1].contourf(
    y.month,
    y.latitude,
    (y.sel(year=1975) - y.sel(year=1870)).transpose("latitude", ...),
    cmap="cmo.balance_r",
    levels=src.utils.make_cb_range(2, 0.2),
    extend="both",
)

for ax in axs:
    ax.set_ylim([-20, 20])

In [ ]:
y = -2e2 * forced["tauy"].sel(longitude=slice(210, 270)).mean("longitude")
# y = 2e2*forced["taux"].sel(longitude=slice(210,270)).mean("longitude")


fig, axs = plt.subplots(2, 1, figsize=(5, 6))
axs[0].contourf(
    y.month,
    y.latitude,
    y.sel(year=1870).transpose("latitude", ...),
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(15, 1.5),
    extend="both",
)

axs[1].contourf(
    y.month,
    y.latitude,
    (y.sel(year=1975) - y.sel(year=1870)).transpose("latitude", ...),
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(1.5, 0.15),
    extend="both",
)

for ax in axs:
    ax.set_ylim([-20, 20])

In [ ]:
y = src.utils.sel_month(Th_total, [2]).stack(s=["member", "time"])

fig, axs = plt.subplots(1, 2, figsize=(6.5, 3))
axs[0].scatter(
    # y["T_3"].sel(year=1870),
    -y["tauy_s"].sel(year=1870),
    (y["pr_n"] - y["pr_n3"]).sel(year=1870),
    s=5,
)

# axs[0].scatter(
#     # y["T_3"].sel(year=1970),
#     -y["tauy_s"].sel(year=1970),
#     (y["pr_n"]-y["pr_n3"]).sel(year=1970),
#     s=5,
# )
# src.utils.set_lims(axs)
plt.show()

### Stratification by season over time

In [ ]:
def get_dTdz_sub(Tsub, mld, delta=25):
    """get velocity at base of mixed layer"""

    ## get temperature difference
    dT = src.utils.get_dT_sub(Tsub=Tsub, Hm=mld, delta=delta)

    ## get gradient
    dTdz = -dT / mld

    return dTdz

In [ ]:
## compute strat
dTdz = get_dTdz_sub(Tsub=forced.T, mld=50, delta=20)

## average over Niño 3
dTdz_n3 = dTdz.sel(longitude=slice(210, 270)).mean("longitude")

## get fractional change
get_frac_change = lambda x: (x - x.isel(year=0)) / x.isel(year=0)


## make plot
fig, ax = plt.subplots(figsize=(2, 4), layout="constrained")

## precip
ax.contourf(
    dTdz_n3.month,
    dTdz_n3.year,
    # (dTdz_n3 - dTdz_n3.isel(year=0)).transpose("year", ...),
    get_frac_change(dTdz_n3).transpose("year", ...),
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(1.5, 0.15),
    extend="both",
)

## formatting
ax.axvline(5, ls="--", lw=0.8, c="k")

plt.show()

$w$ by season

In [ ]:
## compute strat
dTdz = get_dTdz_sub(Tsub=forced.T, mld=50, delta=20)

## average over Niño 3
dTdz_n3 = dTdz.sel(longitude=slice(210, 270)).mean("longitude")

## make plot
fig, ax = plt.subplots(figsize=(2, 4), layout="constrained")

## precip
ax.contourf(
    forced.month,
    forced.year,
    get_frac_change(forced["w_n3"]).transpose("year", ...),
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(0.5, 0.05),
    extend="both",
)

## formatting
ax.axvline(5, ls="--", lw=0.8, c="k")

plt.show()

$w\frac{dT}{dz}$ by season

### Timeseries

In [ ]:
## specify reference coordinate and inflection pt
ref = forced["trop_sst_15"]
# ref = forced["T_3"] - forced["trop_sst_15"]
ref_year = 1980

## specify plot style for reference line
ref_kwargs = dict(ls="--", c="gray", lw=0.8)

#### Tropical SST over time

In [ ]:
sel = lambda x: x.mean("month")
# sel = lambda x : x.sel(month=12)

fig, ax = plt.subplots(figsize=(4, 3))
# ax.plot(forced.year, sel(forced["trop_sst_15"]), label="tropical SST", c="k")
ax.plot(forced.year, sel(ref), label="tropical SST", c="k")
ax.plot(forced.year, sel(forced["T_4"]), label="Niño 4", c="r")
ax.plot(forced.year, sel(forced["T_3"]), label="Niño 3", c="b")
ax.axvline(ref_year, **ref_kwargs)
ax.legend()
plt.show()

#### Mean state quantities, plotted against tropical SST

In [ ]:
sel = lambda x: x.sel(month=[12,1,2]).mean("month")
# sel = lambda x : x.mean("month")

fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(forced.year, sel(forced["T_4"] - forced["T_3"]))
# ax.plot(sel(ref), sel(forced["T_4"] - forced["T_3"]))
# ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.axvline(ref_year, **ref_kwargs)
ax.set_title(r"$\frac{dT}{dx}$")
plt.show()

fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(sel(ref), 8.6e4 * sel(forced["pr_n"] - forced["pr_s"]))
ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.set_title(r"$\frac{dp}{dy}$")
plt.show()

fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(sel(ref), -2e2 * sel(get_nino3_south(forced["tauy"])))
ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.set_title(r"$\tau_y$")
plt.show()


fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(sel(ref), 8.6e4 * sel(forced["pr_n3"]))
ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.set_title(r"precip")
plt.show()

fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(forced.year, sel(forced["w_n3"]))
# ax.plot(sel(ref), sel(forced["w_n3"]))
# ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.axvline(ref_year, **ref_kwargs)
ax.set_title(r"$w$")
plt.show()

fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(forced.year, sel(dTdz_n3))
# ax.plot(sel(ref), sel(dTdz_n3))
# ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.axvline(ref_year, **ref_kwargs)
ax.set_title(r"$\frac{dT}{dz}$")
plt.show()


fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(sel(ref), sel(forced["T_3"] - forced["trop_sst_15"]))
ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.set_title(r"Niño 3 relative SST")
# ax.set_ylim([0,None])
plt.show()

fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(forced.year, sel(forced["trop_sst_15"] - forced["trop_sst_05"]))
# ax.plot(sel(ref), sel(forced["trop_sst_15"] - forced["trop_sst_05"]))
# ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.set_title(r"Niño 3 relative SST")
ax.axvline(ref_year, **ref_kwargs)
# ax.set_ylim([0,None])

plt.show()

#### Variance over time

In [ ]:
## specify which variable to plot
# VARNAME = "eli_30"
VARNAME = "T_3"

## make colormap
CMAP = cmocean.cm.amp
CMAP_NORM = plt.Normalize(vmin=-1, vmax=3)

#### plot variance over time
fig, ax = plt.subplots(figsize=(4, 3))
for i, m_ in enumerate([1, 4, 7, 10]):
    sel_ = lambda x: x.sel(month=m_)
    ax.plot(
        sel(ref),
        sel_(Th_delta_sigma[VARNAME]),
        c=CMAP(CMAP_NORM(i)),
        label=f"Month {m_}",
    )


ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.set_title(r"Niño variance")
# ax.set_ylim([0,None])
plt.show()

fig, ax = plt.subplots(figsize=(4, 3))
for i, m_ in enumerate([1, 4, 7, 10]):
    sel_ = lambda x: x.sel(month=m_)
    ax.plot(
        Th_delta_sigma.year,
        sel_(Th_delta_sigma[VARNAME]),
        c=CMAP(CMAP_NORM(i)),
        label=f"Month {m_}",
    )


# ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.set_title(r"Niño variance")
ax.axvline(ref_year, **ref_kwargs)
ax.axvline(2030, **ref_kwargs)
# ax.set_ylim([0,None])
ax.legend()
plt.show()

PDFs

In [ ]:
## specify varname
# VARNAME="eli_30"
VARNAME = "eli_30"

## get data
x = src.utils.sel_month(Th_total[VARNAME], [12]).stack(s=["member", "time"])


## compute PDFs
edges = np.arange(120, 285, 2)
pdf0, _ = src.utils.get_empirical_pdf(x.sel(year=1870), edges=edges)
pdf1, _ = src.utils.get_empirical_pdf(x.sel(year=2085), edges=edges)

fig, ax = plt.subplots(figsize=(4, 3))
ax.stairs(pdf0, edges)
ax.stairs(pdf1, edges)
ax.set_xlim([180, 215])
plt.show()

In [ ]:
## specify varname
# VARNAME="eli_30"
VARNAME = "pr_n3"

## get data
x = src.utils.sel_month(Th_total[VARNAME], 12).stack(s=["member", "time"])


## compute PDFs
edges = np.arange(0, 30, 0.5)
pdf0, _ = src.utils.get_empirical_pdf(8.6e4 * x.sel(year=1870), edges=edges)
pdf1, _ = src.utils.get_empirical_pdf(8.6e4 * x.sel(year=2085), edges=edges)

fig, ax = plt.subplots(figsize=(4, 3))
ax.stairs(pdf0, edges)
ax.stairs(pdf1, edges)
ax.set_xlim([0, 15])
plt.show()

#### Skewness over time

In [ ]:
## specify which variable to plot
VARNAME = "eli_30"

## make colormap
CMAP = cmocean.cm.amp
CMAP_NORM = plt.Normalize(vmin=-1, vmax=3)

#### plot variance over time
fig, ax = plt.subplots(figsize=(4, 3))
for i, m_ in enumerate([1, 4, 7, 10]):
    sel_ = lambda x: x.sel(month=m_)
    ax.plot(
        # sel(ref),
        Th_delta_skew.year,
        sel_(Th_delta_skew[VARNAME]),
        c=CMAP(CMAP_NORM(i)),
        label=f"Month {m_}",
    )


# ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
ax.axhline(0, **ref_kwargs)
ax.set_title(r"Niño skewness")

ax.legend()

# ax.set_ylim([0,None])
plt.show()

#### Quantiles over time

In [ ]:
## specify which variable to plot
VARNAME = "pr_n3"
SEASON = "JJA"
sel_ = lambda x: x[VARNAME].sel(season=SEASON)
# diff_ = lambda x : x-x.isel(year=0)
# prep = lambda x : diff_(sel_(x))
prep = lambda x: sel_(x)

## specify quantile to plot
Q = 0.95

#### plot variance over time
fig, ax = plt.subplots(figsize=(4, 3))

## warm
ax.plot(
    # sel(ref),
    quant.year,
    prep(quant).sel(q=Q),
    c="r",
)

## median
ax.plot(
    # sel(ref),
    quant.year,
    prep(quant).sel(
        q=0.5,
    ),
    c="k",
)

## cold
ax.plot(
    # sel(ref),
    quant.year,
    prep(quant).sel(q=1 - Q, method="nearest"),
    c="b",
)

# ax.axhline(0, **ref_kwargs)
ax.set_title(r"Niño quantiles")

# ax.legend()

# ax.set_ylim([0,None])
plt.show()

#### Quantile spread over time

In [ ]:
## specify which variable to plot
VARNAME = "pr_n3"
SEASON = "DJF"

## make colormap
CMAP = cmocean.cm.amp
CMAP_NORM = plt.Normalize(vmin=-1, vmax=3)

#### plot variance over time
fig, ax = plt.subplots(figsize=(4, 3))
for i, m_ in enumerate(quant.season):
    # sel_ = lambda x: x.sel(season=m_, q=.95) - x.sel(season=m_, q=.05)
    sel_ = lambda x: x.sel(season=m_)
    ax.plot(
        # sel(ref),
        quant.year,
        sel_(delta_spread[VARNAME]),
        c=CMAP(CMAP_NORM(i)),
        label=f"{m_.values.item()}",
    )


# ax.axvline(sel(ref.sel(year=ref_year)).values.item(), **ref_kwargs)
# ax.axhline(0, **ref_kwargs)
ax.set_title(r"Niño quantiles")

ax.legend()

# ax.set_ylim([0,None])
plt.show()

##### lower or upper width by season

In [ ]:
## specify which variable to plot
VARNAME = "pr_n3"

## compute upper and lower width
width_lower = quant.sel(q=0.5) - quant.sel(q=0.05)
width_upper = quant.sel(q=0.95) - quant.sel(q=0.5)

## make colormap
CMAP = cmocean.cm.amp
CMAP_NORM = plt.Normalize(vmin=-1, vmax=3)

#### plot variance over time
fig, ax = plt.subplots(figsize=(4, 3))
for i, m_ in enumerate(quant.season):
    ax.plot(
        # sel(ref),
        quant.year,
        width_lower[VARNAME].sel(season=m_),
        c=CMAP(CMAP_NORM(i)),
        label=f"{m_.values.item()}",
    )


ax.set_title(r"Niño quantiles")

ax.legend()

# ax.set_ylim([0,None])
plt.show()

##### warm vs. cold

In [ ]:
## specify which variable to plot
VARNAME = "T_34"
# VARNAME = "pr_n3"
# VARNAME = "eli_30"
SEASON = "DJF"

## specify width bound
wb = 0.05


## specify x-variable
X_VAR = quant.year
# X_VAR = sel(ref)

## get func to select data
sel_ = lambda x: x[VARNAME].sel(season=SEASON)

## compute upper and lower width
width_lower = quant.sel(q=0.5) - quant.sel(q=wb)
width_upper = quant.sel(q=1 - wb) - quant.sel(q=0.5)
width_mean = 0.5 * (width_lower + width_upper)

## make colormap
CMAP = cmocean.cm.amp
CMAP_NORM = plt.Normalize(vmin=-1, vmax=3)

#### plot variance over time
fig, ax = plt.subplots(figsize=(4, 3))
ax.plot(X_VAR, sel_(width_lower), c="blue", label="cold")

ax.plot(X_VAR, sel_(width_upper), c="r", label="warm")

ax.plot(X_VAR, sel_(width_mean), c="k")


ax.set_title(r"Distribution width (95% minus median)")
ax.set_ylabel(r"$^{\circ}$C")
ax.legend()


plt.show()